In [2]:
import nltk
nltk.download('stopwords')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jfuentes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
##Needed to get access to mappening.utils.database since this is under mappening.ml
import sys
sys.path.insert(0,'./../..')

from mappening.utils.database import ucla_events_collection, events_ml_collection
import re
import nltk
import string
from tqdm import tqdm
import numpy as np
from nltk.tokenize.moses import MosesTokenizer
from collections import Counter
import pandas as pd


from gensim.models import Phrases
from nltk.corpus import wordnet as wn

Opening the secret door...
Got the .env secrets...
Connected to database!
Got database collections...
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jfuentes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Login manager set up with auth Blueprint!


In [4]:
class PreprocessText:
    def __init__(self, categorizedEvents):
        """categorizedEvents should be a list of dictionaries each corresponding to an event
            X is the tokenized preprocessed text
            Y is the corresponding categories
            phraseMl is the phrase model that can further trained and used
            phrases is a list of all the phrases identified"""
        self.X = []
        self.Y = []
        tokenizer = MosesTokenizer() #tokenizers are basically an advanced split
        for e in categorizedEvents:
            text = e[u'name'] + " " + e[u'description']
            text = tokenizer.tokenize(text)
            text = self.preprocess(text)
            self.X.append(text)
            self.Y.append(e[u'category'])
        self.phraseMl = Phrases(self.X, min_count=3) #learn ml model for phrase
        self.X = list(self.phraseMl[self.X]) #use ml model for phrases
#         self.X = list(self.phraseMl[self.X]) #get triples
        self.phrases = phrases = set([w for doc in self.X for w in doc if '_' in w])
        
    def matchNotX(self, strg, search=re.compile(r'[^!#$%&()*+,-./:;<=>?@\\^_}|{~0123456789]').search):
        """make sure word has something than punctuation"""
        return bool(search(strg)) #make sure word has something other than punctuation

    def preprocess(self, text):
        """Remove all useless words and punct, make lowercase"""
        stoplist = set('for a of the and to in . / '.split())
        stoplist = set(nltk.corpus.stopwords.words('english')) | stoplist | set(string.punctuation)
        return [word.strip(string.punctuation).lower() for word in text if word not in stoplist and self.matchNotX(word)]    
        
    def topBigrams(self, texts, n, tri=False):
        """Other method of getting phrases, currently unused because phrases can be further trained(online) and saved"""
        flatTexts = []
        for text in texts:
            for word in text:
                flatTexts.append(word)
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        trigram_measures = nltk.collocations.TrigramAssocMeasures()
        topAnswers = []
        if tri:
            finder = nltk.collocations.TrigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(trigram_measures.pmi, n)
        else:
            finder = nltk.collocations.BigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(bigram_measures.pmi, n)

In [5]:
def gatherCategorizedEvents():
    """Return panda dataframe of events with category, description, and name"""
    allCategorizedEvents = []
    allEvents = events_ml_collection.find({}, {"category": 1, "description": 1, "name": 1, "hoster": 1, "_id": 0})
    count = 0
    for e in allEvents:
        count += 1
        e['hoster'] = e['hoster']['name']
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    modernEvents = reduceCategories(allCategorizedEvents)
    print count, "total events, learning from the", len(modernEvents), "well categorized events"
    return pd.DataFrame(modernEvents)

## Event Categorization

In [6]:
def someCurrentCategories():
    """Looks at current events for the categories list, to be used if Facebook changes its events in the future"""
    allCategorizedEvents = []
    allEvents = ucla_events_collection.find({}, {"category": 1, "description": 1, "name": 1, "_id": 0})
    for e in allEvents:
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    skTarget = [e['category'] for e in allCategorizedEvents]
    count = sorted(list(set(skTarget)))
    print(count)
    
curListOfCategories = [u'ART', u'CAUSE', u'COMEDY_PERFORMANCE', u'DANCE', u'DRINKS', u'FILM', u'FITNESS', u'FOOD',
                       u'GAMES', u'GARDENING', u'HEALTH', u'LITERATURE', u'MEETUP', u'MUSIC', u'NETWORKING', u'PARTY',
                       u'RELIGION', u'SHOPPING', u'SPORTS', u'THEATER', u'WELLNESS']

In [7]:
def reduceCategories(events):
    """OTHER will be discarded from the training data"""
    categoryMapping = {
        u'BOOK': u'LITERATURE',
        u'COMEDY': u'COMEDY_PERFORMANCE',
        u'CLASS': u'OTHER',
        u'DINING': u'FOOD',
        u'FAMILY': u'OTHER',
        u'FESTIVAL': u'PARTY',
        u'FOOD_TASTING': u'FOOD',
        u'FUNDRAISER': u'CAUSE',
        u'LECTURE': u'OTHER',
        u'MOVIE': u'FILM',
        u'NEIGHBORHOOD': u'OTHER',
        u'NIGHTLIFE': u'OTHER',
        u'RELIGIOUS': u'RELIGION',
        u'VOLUNTEERING': u'CAUSE',
        u'WORKSHOP': u'OTHER'
    }
    
    for e in events:
        category = e['category']
        if category in categoryMapping:
            e['category'] = categoryMapping[category]
    reducedEvents = [e for e in events if e['category'] != u'OTHER']
    return reducedEvents
    

In [8]:
X = gatherCategorizedEvents()

6936 total events, learning from the 2828 well categorized events


## ML Helper Functions

In [9]:
from sklearn.cross_validation import train_test_split

def train(classifier, X, y, trails=25):
    scores = np.zeros(trails)
    for i in tqdm(range(0, trails)):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=i)

        classifier.fit(X_train, y_train)
        scores[i] = (classifier.score(X_test, y_test))
    print "Average Accuracy over %d trials: %s" % (trails, np.mean(scores))
    classifier.fit(X, y)
    return classifier
 

/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
def predictList(vectorizer, classifier, x):
    print(x)
    x = vectorizer.transform(x)
    y_pred = classifier.predict(x)
    print(y_pred)


In [11]:
def predictCategoryProbability(nameVectorizer, detailVectorizer, classifier, X, threshold=.1):
    X_name_transform = nameVectorizer.transform(X['name'])
    X_details_transform = detailVectorizer.transform(X['description'])
    X_total_transform = hstack([X_name_transform, X_details_transform])
    y_pred = classifier.predict_proba(X_total_transform)
    y_categories = []
    
    for i in range(0, X_total_transform.shape[0]):
        current_categories_probabilities = [] #tuple of category name and prob
        
        #create tuple of category, class
        for j in range(0, len(classifier.classes_)):
            current_categories_probabilities.append((classifier.classes_[j], y_pred[i][j]))
        
        current_categories_probabilities.sort(key=lambda x: x[1], reverse=True) #put highest prob categories first
        current_categories = []
        for k, cp in enumerate(current_categories_probabilities):
            if k == 0 or cp[1] > threshold: #ensures at least one cat 
                current_categories.append(cp[0])
        y_categories.append(current_categories)
        
    return y_categories

# Model Making

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
 
# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)

nbModel = MultinomialNB()
nbModel = train(nbModel, X_name_transform, X['category'])

100%|██████████| 25/25 [00:00<00:00, 69.56it/s]

Average Accuracy over 25 trials: 0.5812164073550212


COMBINE NAME AND DESCRIPTION

In [13]:
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
 
# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
nbModel = train(nbModel, X_total_transform, X['category'])

100%|██████████| 25/25 [00:01<00:00, 22.05it/s]


Average Accuracy over 25 trials: 0.5854596888260254


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
param = {'alpha': [.01, .04, .05, .06, .3, .9, 1.5]}
gs = GridSearchCV(nbModel, param)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:2]

STOPLIST 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
param = {'alpha': [.01, .04, .05, .06, .3, .9, 1.5]}
gs = GridSearchCV(nbModel, param)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:2]

Lemming

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from nltk import WordNetLemmatizer, word_tokenize

def lemma_tokenizer(text):
    wn = WordNetLemmatizer()
    tokenizer = word_tokenize
    return [wn.lemmatize(w) for w in tokenizer(text)]

# create the transform
vectorizer = TfidfVectorizer(stop_words='english', analyzer=lemma_tokenizer)

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
param = {'alpha': [.01, .04, .05, .06, .3, .9, 1.5]}
gs = GridSearchCV(nbModel, param)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:2]

NGrams

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

for ngrams in [(1,1), (1,2), (1,3), (2,2), (2,3), (3,3)]:
    # create the transform
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=ngrams)

    # tokenize and build vocab
    X_name_transform = vectorizer.fit_transform(X['name'])
    X_details_transform = vectorizer.fit_transform(X['description'])
    X_total_transform = hstack([X_name_transform, X_details_transform])

    nbModel = MultinomialNB(alpha=.04)
    print(ngrams)
    nbModel = train(nbModel, X_total_transform, X['category'])

Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
          'max_depth': [30,60,90, None]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

Gradient Booster

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

gb = GradientBoostingClassifier()
param = {'n_estimators': [10, 150, 300]}
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

# FINAL PRODUCT

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
nameVectorizer = TfidfVectorizer(stop_words='english')
detailVectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = nameVectorizer.fit_transform(X['name'])
X_details_transform = detailVectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

rf = RandomForestClassifier(n_estimators=150, max_depth=None)
rf.fit(X_total_transform, X['category'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [30]:
import cPickle

with open(r"categorizationModel.pickle", "wb") as output_file:
    cPickle.dump(rf, output_file)
    
with open(r"nameVectorizer.pickle", "wb") as output_file:
    cPickle.dump(nameVectorizer, output_file)
    
with open(r"detailVectorizer.pickle", "wb") as output_file:
    cPickle.dump(detailVectorizer, output_file)

In [31]:
with open(r"categorizationModel.pickle", "r") as model:
    rfLoaded = cPickle.load(model)
    
with open(r"nameVectorizer.pickle", "r") as model:
    nameVLoaded = cPickle.load(model)
    
with open(r"detailVectorizer.pickle", "r") as model:
    detailVLoaded = cPickle.load(model)

x = predictCategoryProbability(nameVectorizer, detailVectorizer, rfLoaded, X_total_transform)

In [18]:
x

[[u'COMEDY_PERFORMANCE', u'MUSIC'],
 [u'FILM'],
 [u'THEATER'],
 [u'NETWORKING'],
 [u'THEATER'],
 [u'FOOD'],
 [u'LITERATURE'],
 [u'THEATER'],
 [u'FOOD'],
 [u'THEATER', u'FILM'],
 [u'THEATER'],
 [u'ART', u'MUSIC'],
 [u'THEATER'],
 [u'MEETUP'],
 [u'MUSIC'],
 [u'CAUSE'],
 [u'FOOD'],
 [u'CAUSE'],
 [u'MUSIC'],
 [u'NETWORKING'],
 [u'COMEDY_PERFORMANCE'],
 [u'FOOD', u'CAUSE'],
 [u'PARTY', u'FOOD'],
 [u'FOOD'],
 [u'SPORTS'],
 [u'FOOD'],
 [u'CAUSE'],
 [u'PARTY'],
 [u'CAUSE'],
 [u'MUSIC'],
 [u'CAUSE'],
 [u'ART'],
 [u'WELLNESS'],
 [u'MEETUP'],
 [u'FOOD'],
 [u'MUSIC'],
 [u'SPORTS'],
 [u'HEALTH'],
 [u'PARTY'],
 [u'FOOD'],
 [u'CAUSE'],
 [u'CAUSE'],
 [u'CAUSE'],
 [u'THEATER'],
 [u'CAUSE', u'FOOD'],
 [u'SPORTS'],
 [u'ART'],
 [u'THEATER'],
 [u'CAUSE', u'FOOD'],
 [u'MUSIC'],
 [u'FOOD'],
 [u'MEETUP'],
 [u'SPORTS'],
 [u'MEETUP', u'FILM'],
 [u'CAUSE'],
 [u'SPORTS'],
 [u'MEETUP'],
 [u'MEETUP'],
 [u'THEATER'],
 [u'SPORTS'],
 [u'SPORTS'],
 [u'THEATER', u'FILM'],
 [u'THEATER', u'CAUSE'],
 [u'CAUSE'],
 [u'FILM']

In [ ]:
X['description'][0]

In [ ]:
X[2:3]

In [27]:
allEvents = ucla_events_collection.find()
events = [e for e in allEvents]

In [28]:
pd.DataFrame(events)

,_id,attending_count,category,cover,description,duplicate_occurrence,end_time,hoster,id,interested_count,is_canceled,maybe_count,name,noreply_count,place,start_time,ticket_uri,time_updated
0,5a94bc69ccc4e90001e29b73,0,COMEDY_PERFORMANCE,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Every Friday night, these Mainstage teams take...",True,2018-08-24T23:00:00-0700,"{u'id': u'148341023547', u'name': u'The Improv...",1796721490633033,0,False,0,Friday Night Mainstage,0,"{u'id': u'148341023547', u'name': u'The Improv...",2018-08-24T20:00:00-0700,https://www.eventbrite.com/e/the-improv-space-...,2018-04-13 19:10:14.276351
1,5a94bc69ccc4e90001e29b74,0,COMEDY_PERFORMANCE,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Every Friday night, these Mainstage teams take...",True,2018-11-16T23:00:00-0800,"{u'id': u'148341023547', u'name': u'The Improv...",1796721440633038,1,False,1,Friday Night Mainstage,0,"{u'id': u'148341023547', u'name': u'The Improv...",2018-11-16T20:00:00-0800,https://www.eventbrite.com/e/the-improv-space-...,2018-04-13 19:10:14.276710
2,5a94bc6dccc4e90001e29b9e,198,FILM,{u'source': u'https://scontent.xx.fbcdn.net/v/...,Roarrr,NaN,2019-02-19T22:30:00-0800,"{u'id': u'133529946697474', u'name': u'UCLA - ...",177410356341901,1168,False,1168,Very Wild Animal,64,"{u'id': u'133529946697474', u'name': u'UCLA - ...",2019-02-19T19:30:00-0800,NaN,2018-04-13 19:10:14.277055
3,5a95c9412c53ef000162134c,9,NaN,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Whether you’re a student, working developer, o...",NaN,2018-04-29T17:00:00-0700,"{u'id': u'256011207748013', u'name': u'UCLA Ca...",336029530140276,69,False,69,Okta Hackathon,6,"{u'location': {u'city': u'Los Angeles', u'zip'...",2018-04-28T10:00:00-0700,NaN,2018-04-13 19:10:14.277415
4,5a94bc32ccc4e90001e2991b,0,COMEDY_PERFORMANCE,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Every Friday night, these Mainstage teams take...",True,2018-04-27T23:00:00-0700,"{u'id': u'148341023547', u'name': u'The Improv...",1796721547299694,0,False,0,Friday Night Mainstage,0,"{u'id': u'148341023547', u'location': {u'city'...",2018-04-27T20:00:00-0700,https://www.eventbrite.com/e/the-improv-space-...,2018-04-13 19:10:14.277729
5,5ab06db342302f00016899c3,0,FILM,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Dir. Katy Lena Ndiaye, 2007, 56 min.\n\nAwaiti...",NaN,2018-04-18T20:00:00-0700,"{u'id': u'80305275655', u'name': u'Fowler Muse...",222640314959902,3,False,3,Screening: Awaiting for Men,0,"{u'location': {u'city': u'Los Angeles', u'zip'...",2018-04-18T19:00:00-0700,https://www.fowler.ucla.edu/visit/,2018-04-13 19:10:14.278046
6,5ab5b3ae42302f0001689a1c,0,NaN,{u'source': u'https://scontent.xx.fbcdn.net/v/...,Hey Transfers! If you need help with writing p...,True,2018-05-22T17:00:00-0700,"{u'id': u'401227390026546', u'name': u'UCLA Tr...",178252446131506,0,False,0,Undergraduate Writing Center Office Hours,0,"{u'location': {u'city': u'Los Angeles', u'zip'...",2018-05-22T12:00:00-0700,NaN,2018-04-13 19:10:14.278364
7,5ab2bc4042302f00016899ea,0,NaN,{u'source': u'https://scontent.xx.fbcdn.net/v/...,Hey Transfers! If you need help with writing p...,True,2018-05-21T17:00:00-0700,"{u'id': u'401227390026546', u'name': u'UCLA Tr...",177708689519215,0,False,0,Undergraduate Writing Center Office Hours,0,"{u'id': u'401227390026546', u'location': {u'ci...",2018-05-21T12:00:00-0700,NaN,2018-04-13 19:10:14.278672
8,5a94bc56ccc4e90001e29a8e,0,COMEDY_PERFORMANCE,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Every Friday night, these Mainstage teams take...",True,2018-08-17T23:00:00-0700,"{u'id': u'148341023547', u'name': u'The Improv...",1796721493966366,0,False,0,Friday Night Mainstage,0,"{u'id': u'148341023547', u'location': {u'city'...",2018-08-17T20:00:00-0700,https://www.eventbrite.com/e/the-improv-space-...,2018-04-13 19:10:14.278965
9,5a94bc57ccc4e90001e29a9c,0,COMEDY_PERFORMANCE,{u'source': u'https://scontent.xx.fbcdn.net/v/...,"Every Friday night, these Mainstage teams take...",True,201

In [32]:
allEvents

In [34]:
events

[{u'_id': ObjectId('5a94bc69ccc4e90001e29b73'),
  u'attending_count': 0,
  u'category': u'COMEDY_PERFORMANCE',
  u'cover': {u'id': u'10156017553558548',
   u'offset_x': 0,
   u'offset_y': 2,
   u'source': u'https://scontent.xx.fbcdn.net/v/t31.0-8/p720x720/27983075_10156017553558548_2574020675235322908_o.jpg?oh=94c51bd535c9c5f9e21c2c72e4c9d09a&oe=5B14226E'},
  u'description': u"Every Friday night, these Mainstage teams take the stage from 8-11 PM. Your $5 gets you in for the entire night. Bring your friends because every hour is a real 'you had to be there'.\n\n8PM\n\nHOT FOR THE RENT\nCorey Blake, Alex Knight, Katie Goldston, Dewey Cassaday, Phoebe Lim, Jane Wilson, and Cory Peter Lane\n\nSTOP, ROCK, AND ROLL\nElisa Espinoza, James Gangl, Dianna Lee, Liz Valoon, Sam Stefanak, Makela Yepez, Alex Wolfson, and Mitchell Mack.\n\n\n9PM\n\nBANKROLL\nNikhil Deshpande, Nick Eliafan, Bailey Kai, Hunter Saling, Annabelle Seymour, Will Stephens, Kevin Alvarez, and Tesha Kondrat.\n\nTHE 400 CAT\nG

In [48]:
import cPickle as pickle
import pandas as pd

# Needed to get access to mappening.utils.database when running just this file since this is under mappening.ml
import sys
sys.path.insert(0,'./../..')

from mappening.utils.database import events_ml_collection

def categorizeEvents(events):
    """
    :param X: should be list of dictionary elements
    Returns list of events updated with a list of categories
    """

    #make copy just for machine learning
    #TODO: could be optimized by just changing the original events then running through and deleting empty string values
    X = events[:]
    for event in X:
        if 'name' not in event:
            event['name'] = ''
        if 'description' not in event:
            event['description'] = ''

    # Load data
    X = pd.DataFrame(events)
    with open(r"categorizationModel.pickle", "r") as model:
        rf = pickle.load(model)
    with open(r"nameVectorizer.pickle", "r") as model:
        nameVectorizer = pickle.load(model)
    with open(r"detailVectorizer.pickle", "r") as model:
        detailVectorizer = pickle.load(model)

    catLists = predictCategoryProbability(nameVectorizer, detailVectorizer, rf, X)

    for i in range(0, len(catLists)):
        events[i]['category'] = catLists[i]
    return events

def predictCategoryProbability(nameVectorizer, detailVectorizer, classifier, X, threshold=.1):
    """
    :param nameVectorizer: TfidfVectorizer for the event names
    :param detailVectorizer: TfidfVectorizer for details
    :param classifer: scikit classifer with predict probability function(e.g RandomForestClassifier)
    :param X: pandas dataframe with 'name' and 'description' columns
    :param threshold: probabilty threshold for classifer prediction(note depending on classifer p varies)

    Returns parallel list of categories
    """

    X_name_transform = nameVectorizer.transform(X['name'])
    X_details_transform = detailVectorizer.transform(X['description'])
    X_total_transform = hstack([X_name_transform, X_details_transform])
    y_pred = classifier.predict_proba(X_total_transform)
    y_categories = []

    #create list of categories of len > 0 or more if over threshold
    for i in range(0, X_total_transform.shape[0]):
        current_categories_probabilities = [] #tuple of category name and prob

        #create tuple of category, class
        for j in range(0, len(classifier.classes_)):
            current_categories_probabilities.append((classifier.classes_[j], y_pred[i][j]))

        current_categories_probabilities.sort(key=lambda x: x[1], reverse=True) #put highest prob categories first
        current_categories = []
        for k, cp in enumerate(current_categories_probabilities):
            if k == 0 or cp[1] > threshold: #ensures at least one cat
                current_categories.append(cp[0])
        y_categories.append(current_categories)

    return y_categories

allEventsIterator = events_ml_collection.find({}, {"category": 1, "description": 1, "name": 1, "hoster": 1, "_id": 0})
allEvents = [e for e in allEventsIterator]
x = categorizeEvents(allEvents)


In [37]:
allEventsIterator = events_ml_collection.find({}, {"category": 1, "description": 1, "name": 1, "hoster": 1, "_id": 0})
allEvents = [e for e in allEventsIterator if 'description' not in e]

In [49]:
allEvents

[{'category': [u'CAUSE', u'MEETUP', u'PARTY'],
  u'description': u'Hey hey Circle K! As we\'ve announced the past week, our Spring Member Retreat facilitated by our club\'s seniors is finally here! Come out for fun games and bonding activities and get to know more about our seniors\' experiences before they graduate. We will be using this page for important updates before and during the event, so click "Going" to stay tuned!\n\nDIRECTIONS TO EVENT LOCATION (Sunset Rec Mesa Lawn):\nEnter Sunset Rec from either behind Hedrick Summit or from De Neve Drive (the direction of the Sunset Village tennis courts).\nGo to the pool area in the middle of Sunset Rec.\nFrom the pool, there is a trail leading up the hill, marked by a sign saying "Mesa Room." Go up this trail and you\'re there!\n\nIf you are lost or have any questions, please contact:\nAlex Levine // 408 425 4754',
  u'hoster': {u'id': u'2200028282',
   u'name': u'UCLA Circle K International (CKI)'},
  u'name': u'Circle K Spring Member

In [50]:
x = [{'hi': 'there'}, {'my': 'name'}]
for e in x:
    if 'hi' not in e:
        e['hi'] = ''


In [51]:
x

[{'hi': 'there'}, {'hi': '', 'my': 'name'}]

In [62]:
[x[0].get('bo')] + []

[None]

In [58]:
[None] == []

False